In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_matching_tools.io.graph_dataset import GraphDataset
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
from graph_matching_tools.algorithms.multiway.hippi import hippi_multiway_matching
from graph_matching_tools.algorithms.kernels.gaussian import create_gaussian_node_kernel
from graph_matching_tools.algorithms.kernels.utils import create_full_node_affinity_matrix
from graph_matching_tools.algorithms.multiway.stiefel import sparse_stiefel_manifold_sync
from graph_matching_tools.algorithms.multiway import quickmatch
from graph_matching_tools.algorithms.multiway import matcheig
import tools.graph_processing as gp
import scipy.io as sio

In [2]:
def get_permutation_matrix_from_dictionary(matching, g_sizes):
    """
    Create the full permutation matrix from the matching result
    :param matching: the matching result for each graph (nodes number, assignment)
    :param g_sizes: the list of the size of the different graph
    :return: the full permutation matrix
    """
    f_size = int(np.sum(g_sizes))
    res = np.zeros((f_size, f_size))

    idx1 = 0
    for i_g1 in range(len(g_sizes)):
        idx2 = 0
        for i_g2 in range(len(g_sizes)):
            match = matching["{},{}".format(i_g1, i_g2)]
            for k in match:
                res[idx1 + int(k), idx2 + match[k]] = 1
            idx2 += g_sizes[i_g2]
        idx1 += g_sizes[i_g1]
        
    np.fill_diagonal(res,1)
    return res

In [3]:
def remove_dummy_rowcol_X(bulk_X, graphs):
    
    dummy_mask = [list(nx.get_node_attributes(graph,'is_dummy').values()) for graph in graphs]
    dummy_mask = sum(dummy_mask,[])
    dummy_indexes = [i for i in range(len(dummy_mask)) if dummy_mask[i]==True]     
    
    bulk_X = np.delete(bulk_X,dummy_indexes,0) # delete the dummy rows
    bulk_X = np.delete(bulk_X,dummy_indexes,1) # delete the dummy columns
    
    return bulk_X

In [4]:
#path_to_graph_folder = '../data/simu_graph/NEW_SIMUS_JULY_11/'
path_to_dummy_graphs_folder = '../data/Oasis_original_new_with_dummy/'

In [5]:
sigma = 200

udim = 101

                
#path_to_graphs = path_to_graph_folder + '/' + trial + '/' + folder+'/graphs/'
path_to_dummy_graphs = path_to_dummy_graphs_folder +'/modified_graphs/'


list_graphs_dummy = gp.load_graphs_in_list(path_to_dummy_graphs)
sizes_dummy = [nx.number_of_nodes(g) for g in list_graphs_dummy]

path_X_kergm = path_to_dummy_graphs_folder + '/X_kerGM.mat' 
knode = sio.loadmat(path_X_kergm)['full_assignment_mat']
print('X_KerGM shape', knode.shape)

P = matcheig.matcheig(knode, udim, sizes_dummy)

X_MatchEig = {}
X_MatchEig['X'] = P

print('P shape', P.shape)

#save with dummy
sio.savemat(path_to_dummy_graphs_folder + '/' + '/X_MatchEig.mat',X_MatchEig,do_compression=True)


X_KerGM shape (13837, 13837)
P shape (13837, 13837)
